In [55]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [62]:
# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone

# --------- #
# LEAVE ME  #
# --------- #
df_main = df_daily[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope'])


# Run Train_Flow

In [83]:
import train_flow
import importlib
importlib.reload(train_flow)

# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['exclude_new', 'include_new', 'run_separately'] #'exclude_new', 'include_new', 'run_separately'
train_flow.pi_handling_test(pi_handlings)

"""
Applicable when appending new features to an existing set
run_separately: runs pi for old and new feature set separately --> combines the top features from each set at end
exclude_new: runs pi only for old feature set then appends all new features onto the trimmed list --> combines the top old list features with all new features 
include_new: runs pi for combined old and new feature set --> if new features aren't adding incremental value they will be completely excluded
"""


# ---------- #
# UPDATE ME  #
# ---------- #
new_features = past_ret_cols #past_sum_cols
target_horizons = [2, 5, 10, 20, 30]
train_years = [5]
pi_years = [1.5]
min_feats = [8]
days_assessed = 120
test_day = 1
feature_cols = feature_dict['ma_lag'] #feature_cols
list_name = "ma_lag+pr10"


# --------- #
# LEAVE ME  #
# --------- # 
models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
results_df, perm_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                   train_years, pi_years, min_feats, list_name, new_features)

Running for horizon 2 | exclude_new
74 | 22 | ['Past_Return%_1', 'Past_Return%_10', 'Past_Return%_2', 'Past_Return%_20', 'Past_Return%_3', 'Past_Return%_30', 'Past_Return%_5', 'Past_Return_10_sum10', 'Past_Return_1_sum10', 'Past_Return_20_sum10', 'Past_Return_2_sum10', 'Past_Return_30_sum10', 'Past_Return_3_sum10', 'Past_Return_5_sum10', 'SMA_10_Lag100_min', 'SMA_10_Lag10_max', 'SMA_10_Lag10_min', 'SMA_10_Lag150_min', 'SMA_10_Lag50_max', 'SMA_10_Lag50_min', 'SMA_25_Lag25_min', 'SMA_25_Lag50_min']
Run 1/120 | Train: 2021-03-17 → 2026-01-30 | Test: 2026-02-04 → 2026-02-04 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)
74 | 22 | ['Past_Return%_1', 'Past_Return%_10', 'Past_Return%_2', 'Past_Return%_20', 'Past_Return%_3', 'Past_Return%_30', 'Past_Return%_5', 'Past_Return_10_sum10', 'Past_Return_1_sum10', 'Past_Return_20_sum10', 'Past_Return_2_sum10', 'Past_Return_30_sum10', 'Past_Return_3_sum10', 'Past_Return_5_sum10', 'SMA_10_Lag100_min', 'SMA_10_Lag10_min', 'SMA_10_Lag150_min', 'S

# Save most recent run

In [84]:
# ---------- #
# UPDATE ME  #
# ---------- #
df = pd.read_csv("baseline_all.csv")


# --------- #
# LEAVE ME  #
# --------- #
cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon',
        'pi_size', 'min_feats']
df_new = results_df[cols].copy()
df_concat = pd.concat([df[cols], df_new], ignore_index=True)


# ---------- #
# UPDATE ME  #
# ---------- #
df_concat.to_csv('baseline_new.csv', index=False)

# Run Performance_Flow

In [90]:
import performance_flow
importlib.reload(performance_flow)

# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "baseline_new.csv" # Match prior cell saved as file name
min_th = 0.55
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)

In [95]:
composite_score[composite_score["horizon"] == 30]

,horizon,model,train_years,feature_set,pi_size,min_feats,pos_rate,records,bal_prec,bal_prec_|0.75|,brier,log_loss,mcc,pprec,nprec,cov_|0.75|,pprec_|0.75|,nprec_|0.75|,composite
319,30,xgboost-3,5,ma_lag+pr10,1.5-i,8,0.84,116.0,0.69,0.70,0.20,3.90,0.47,0.96,0.41,0.96,0.96,0.43,0.63
129,30,xgboost-3,6,ma_sma_ba,1.5,100,0.83,117.0,0.69,0.67,0.23,5.00,0.49,0.99,0.39,0.94,0.98,0.36,0.63
229,30,xgboost-3,6,ma_sma_ba,1.5,6,0.84,118.0,0.68,0.71,0.18,2.42,0.45,0.95,0.41,0.90,0.97,0.44,0.62
128,30,xgboost-3,4,ma_sma_ba,1.5,100,0.85,113.0,0.67,0.66,0.24,5.49,0.47,0.99,0.36,0.95,0.98,0.34,0.61
20,30,xgboost-3,6,ma_sma-baseline,1.5,8,0.83,230.0,0.67,0.67,0.24,5.10,0.45,0.97,0.38,0.97,0.97,0.36,0.61
324,30,xgboost-3,5,ma_lag+pr10,1.5-r,8,0.82,120.0,0.67,0.67,0.21,4.83,0.41,0.92,0.43,0.96,0.93,0.41,0.60
314,30,xgboost-3,5,ma_lag+pr10,1.5-e,8,0.83,116.0,0.66,0.67,0.21,4.17,0.40,0.93,0.40,0.97,0.94,0.41,0.59
78,30,xgboost-3,4,ma_sma-baseline,1.5,8,0.84,115.0,0.65,0.65,0.26,5.19,0.41,0.97,0.33,0.92,0.97,0.33,0.58
72,30,xgboost-3,4,ma_lag-baseline,1.5,8,0.83,119.0,0.64,0.64,0.26,7.13,0.36,0.93,0.35,0.96,0.93,0.35,0.56
109,30,xgboost-3,6,ma_lag_ba,1.5,100,0.83,120.0,0.64,0.64,0.24,6.01,0.34,0.92,0.36,0.98,0.91,0.36,0.56
